<a href="https://colab.research.google.com/github/ArkanDash/Advanced-RVC-Inference/blob/master/Advanced-RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RVC Inference

A modular Retrieval-based Voice Conversion framework with Gradio UI, training capabilities, and audio processing tools

[Discord](https://discord.gg/hvmsukmBHE) - [Github](https://github.com/ArkanDash/Advanced-RVC-Inference)

In [ ]:
# @title Mount Drive
from google.colab import drive
from google.colab._message import MessageError

try:
  drive.mount("/content/drive")
except MessageError:
  print("❌ Failed to mount drive")

In [ ]:
# @title Setup runtime environment
# markdown ### Choose installation method:
install_method = "local"

from IPython.display import clear_output
import os

# Define paths first
LOGS_PATH = "/content/Advanced-RVC-Inference/advanced_rvc_inference/assets/logs"
BACKUPS_PATH = f"/content/drive/MyDrive/RVCBackup"

# Install system dependencies for audio processing
!apt-get -y install libportaudio2 ffmpeg git -qq > /dev/null 2>&1

# Install uv for faster package management
!pip install uv -q

if install_method == 'local':
    # Clone from git and install locally (Recommended for development)
    if os.path.exists("Advanced-RVC-Inference"):
        %cd Advanced-RVC-Inference
        !git pull origin master
    else:
        !git clone https://github.com/ArkanDash/Advanced-RVC-Inference.git
        %cd Advanced-RVC-Inference

    # Install from requirements.txt
    !uv pip install -r requirements.txt --system -q

    # Install package in editable mode
    !uv pip install -e . --system -q

elif install_method == 'git+uv':
    # Install latest version from GitHub with uv (fastest)
    !uv pip install git+https://github.com/ArkanDash/Advanced-RVC-Inference.git -q


clear_output()
print("Finished installing Advanced RVC Inference!")
print("Use the cell below to start the server.")

In [ ]:
# @title **Start server**
# @markdown ### Choose a sharing method:
method = "gradio"  # @param ["gradio", "localtunnel", "ngrok"]
ngrok_token = ""  # @param {type:"string"}
from IPython.display import clear_output
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

# Additional environment variables for better performance
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Change to project directory if using local installation
if install_method == 'local':
    repo_dir = "Advanced-RVC-Inference"
    # Check if we're already in the repo directory
    if os.path.basename(os.getcwd()) != repo_dir:
        if os.path.exists(repo_dir):
            %cd $repo_dir
            !git pull -q

clear_output()
print("Starting Advanced RVC Inference Server...\n")

if method == 'gradio':
    # Start with share option
    print("Starting Gradio server with public link...")
    !rvc-gui --share

elif method == 'localtunnel':
    !pip install localtunnel -q
    clear_output()
    print("Starting server with localtunnel...")
    print("\nPublic IP: $(curl -s ifconfig.me)")
    print("\nStarting localtunnel in background...")
    # Start localtunnel and RVC server in parallel
    !lt --port 7869 --subdomain rvc-advanced &> /tmp/lt.log &
    !sleep 5
    !rvc-gui --listen --server-name 0.0.0.0

elif method == 'ngrok':
    !pip install pyngrok -q
    from pyngrok import ngrok, conf

    # Kill existing ngrok tunnels
    ngrok.kill()

    if ngrok_token:
        ngrok.set_auth_token(ngrok_token)

    clear_output()
    print("Starting server with ngrok...")

    # Create ngrok tunnel
    public_url = ngrok.connect(7869)
    print(f"\nNgrok URL: {public_url}")

    # Start RVC server
    !rvc-gui --listen --server-name 0.0.0.0 --server-port 7869

# Real-time monitoring and backup service (runs in background)
print("\n\nStarting real-time backup service...")
print("This service will monitor for model changes and auto-backup.")

# Create a simple monitoring script
monitor_script = '''
#!/usr/bin/env python3
import os
import time
import json
from pathlib import Path
from datetime import datetime
import shutil

LOGS_PATH = "/content/Advanced-RVC-Inference/advanced_rvc_inference/assets/logs"
BACKUPS_PATH = "/content/drive/MyDrive/RVCBackup"
CONFIG_FILE = os.path.join(BACKUPS_PATH, "backup_config.json")

def load_config():
    if os.path.exists(CONFIG_FILE):
        with open(CONFIG_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_config(config):
    with open(CONFIG_FILE, 'w') as f:
        json.dump(config, f, indent=2)

def backup_modified_models():
    config = load_config()
    if not config.get('auto_backup', True):
        return

    logs_path = Path(LOGS_PATH)
    backup_path = Path(BACKUPS_PATH)

    if not logs_path.exists() or not backup_path.exists():
        return

    for model_dir in logs_path.iterdir():
        if model_dir.is_dir() and model_dir.name not in ['mute', 'reference', 'zips']:
            backup_dir = backup_path / model_dir.name

            # Check if model was modified
            if not backup_dir.exists() or \
               model_dir.stat().st_mtime > backup_dir.stat().st_mtime:
                try:
                    if backup_dir.exists():
                        shutil.rmtree(backup_dir)
                    shutil.copytree(model_dir, backup_dir)
                    print(f"[Auto-Backup] Backed up {model_dir.name}")
                except Exception as e:
                    print(f"[Auto-Backup] Failed to backup {model_dir.name}: {e}")

    config['last_backup'] = datetime.now().isoformat()
    save_config(config)

if __name__ == "__main__":
    print("Real-time backup monitor started...")
    print(f"Monitoring: {LOGS_PATH}")
    print(f"Backup to: {BACKUPS_PATH}")

    while True:
        try:
            backup_modified_models()
            time.sleep(60)  # Check every minute
        except KeyboardInterrupt:
            print("\nBackup monitor stopped.")
            break
        except Exception as e:
            print(f"Monitor error: {e}")
            time.sleep(60)
'''

# Write and start the monitor script
with open('/tmp/backup_monitor.py', 'w') as f:
    f.write(monitor_script)

# Start the monitor in the background
!python /tmp/backup_monitor.py > /tmp/backup_monitor.log 2>&1 &

print("\n✅ Backup service started. Check /tmp/backup_monitor.log for details.")

# BackUp Model

In [ ]:




# @markdown ### Enter the name of the model to backup:
model_to_load = "" # @param {type:"string"}

import os
import shutil
from google.colab import drive
from google.colab._message import MessageError

# Mount Google Drive if not already mounted
try:
  drive.mount("/content/drive")
  print("✅ Google Drive mounted successfully.")
except MessageError:
  print("❌ Failed to mount drive. It might already be mounted.")
except Exception as e:
  print(f"An unexpected error occurred while mounting drive: {e}")

# Define paths (using values from kernel state for consistency)
LOGS_PATH = f"/content/Advanced-RVC-Inference/advanced_rvc_inference/assets/logs/{model_to_load}"
BACKUPS_PATH = "/content/drive/MyDrive/RVCBackup"

print(f"\nAttempting to backup logs from: {LOGS_PATH}")
print(f"To Google Drive path: {BACKUPS_PATH}")

if not os.path.exists(LOGS_PATH):
    print(f"❌ Error: Source logs path '{LOGS_PATH}' does not exist.")
else:
    os.makedirs(BACKUPS_PATH, exist_ok=True)
    print(f"Ensured backup directory '{BACKUPS_PATH}' exists.")

    backed_up_count = 0
    # Iterate through each model directory in LOGS_PATH
    for item in os.listdir(LOGS_PATH):
        source_item_path = os.path.join(LOGS_PATH, item)
        destination_item_path = os.path.join(BACKUPS_PATH, item)

        # Exclude common non-model log directories
        if os.path.isdir(source_item_path) and item not in ['mute', 'reference', 'zips']:
            print(f"Backing up model logs for '{item}'...")
            try:
                if os.path.exists(destination_item_path):
                    shutil.rmtree(destination_item_path) # Remove existing backup to ensure fresh copy
                shutil.copytree(source_item_path, destination_item_path)
                print(f"✅ Successfully backed up '{item}' logs to '{destination_item_path}'")
                backed_up_count += 1
            except Exception as e:
                print(f"❌ Failed to backup '{item}' logs: {e}")

    if backed_up_count == 0:
        print("ℹ️ No eligible model log directories were found or backed up.")
    else:
        print(f"\n✅ All eligible model logs from '{LOGS_PATH}' have been backed up to '{BACKUPS_PATH}'.")

### Load Backup Models from Google Drive

Enter the name of the model you wish to load from your Google Drive backups. This will copy the model's log data from `RVCBackup` in your Drive back to the `advanced_rvc_inference/assets/logs` directory in the Colab environment.

In [ ]:
import os
import shutil
from google.colab import drive
from google.colab._message import MessageError

# Mount Google Drive if not already mounted
try:
  drive.mount("/content/drive")
  print("✅ Google Drive mounted successfully.")
except MessageError:
  print("❌ Failed to mount drive. It might already be mounted.")
except Exception as e:
  print(f"An unexpected error occurred while mounting drive: {e}")

# Define paths (using values from kernel state for consistency)
LOGS_PATH = "/content/Advanced-RVC-Inference/advanced_rvc_inference/assets/logs"
BACKUPS_PATH = "/content/drive/MyDrive/RVCBackup"

print(f"\nAvailable models in backup directory ('{BACKUPS_PATH}'):")
if not os.path.exists(BACKUPS_PATH):
    print(f"❌ Backup directory '{BACKUPS_PATH}' does not exist. No backups found.")
else:
    backup_models = [d for d in os.listdir(BACKUPS_PATH) if os.path.isdir(os.path.join(BACKUPS_PATH, d))]
    if backup_models:
        for i, model_name in enumerate(backup_models):
            print(f"  {i+1}. {model_name}")
    else:
        print("  No model backups found in the specified directory.")


# @markdown ### Enter the name of the model to load:
model_to_load = "" # @param {type:"string"}

if not model_to_load:
    print("Please enter a model name to load.")
elif model_to_load not in backup_models:
    print(f"❌ Model '{model_to_load}' not found in '{BACKUPS_PATH}'. Please check the name and try again.")
else:
    source_path = os.path.join(BACKUPS_PATH, model_to_load)
    destination_path = os.path.join(LOGS_PATH, model_to_load)

    print(f"\nAttempting to load model '{model_to_load}' from '{source_path}'")
    print(f"To Colab environment path: '{destination_path}'")

    try:
        if os.path.exists(destination_path):
            shutil.rmtree(destination_path) # Remove existing model data in logs to avoid conflicts
            print(f"Existing model data at '{destination_path}' removed.")

        shutil.copytree(source_path, destination_path)
        print(f"✅ Successfully loaded model '{model_to_load}' to '{destination_path}'.")
        print("You may need to refresh the RVC GUI or restart the server to see the loaded model.")
    except Exception as e:
        print(f"❌ Failed to load model '{model_to_load}': {e}")

# Push The Model

In [ ]:

#@title  <small> - Join thousands of model makers by uploading your model to huggingface! All you need is your token. https://huggingface.co/settings/tokens
%cd /content
import os
from google.colab import userdata
from huggingface_hub import HfApi, create_repo, login

HF_TOKEN = "" #@param {type:"string"}
login(f"{HF_TOKEN}")
api = HfApi()
#@markdown Your REPO_ID is the name of your username + whatever you want to name the folder where it will be saved  🤗
repo_id = "NeoPy/My-Voice" #@param {type:"string"}
model_name = "" #@param {type:"string"}
model_path = f"/content/Advanced-RVC-Inference/advanced_rvc_inference/assets/logs/{model_name}"

try:
    create_repo(repo_id)
except:
    print("Error creating repo, proceeding...")

api.upload_file(
    path_or_fileobj=f"{model_path}/{model_name}.zip",
    path_in_repo=f'{model_name}.zip',
    repo_id=repo_id,
    repo_type = "model"
)